# 목적

VGG19_Pure가 전혀 학습이 되지 않고 있다. 왜일까
저거도 원인을 찾아야 할 것은데, 우선 pretrained된 VGG19를 가져오는 방식으로도 구현해보자.

# 0.1. 구글 드라이브 연동

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# 0.2. 라이브러리 임포트 및 전역변수 설정

In [2]:
from pathlib import Path
from easydict import EasyDict as edict
from PIL import Image
from tqdm import tqdm
from torchvision import transforms
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset, random_split
from collections import Counter
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
root = '/gdrive/My Drive/ctp431'
inst_pool = ['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']

# Step 1. VGG19 모델 만들기

In [12]:
class VGG19_Pretrained(nn.Module):
    def __init__(self, num_class = 11):
        super(VGG19_Pretrained, self).__init__()

        pretrained_vgg = models.vgg19(pretrained=True)

        # for param in pretrained_vgg.features.parameters():
        #     param.requires_grad = False

        self.features = pretrained_vgg.features

        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Step 2. 학습 돌리기

In [13]:
# 데이터 로드
test_data_path = root + "/MK2/DataSet/test_data.pt"
test_x, test_y = torch.load(test_data_path)

train_data_path = root + "/MK2/DataSet/training_data.pt"
train_x, train_y = torch.load(train_data_path)

# 데이터셋과 DataLoader 생성
train_data = TensorDataset(train_x, train_y)
test_data = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_data, batch_size=35, shuffle=True)
val_loader = DataLoader(test_data, batch_size=35, shuffle=False)

# 클래스 분포 확인
print(train_y.unique(), train_y.bincount())
print(test_y.unique(), test_y.bincount())

<ipython-input-13-8f30f2df9be4>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_x, test_y = torch.load(test_data_path)
<ipython-input-13-8f30f2df9be4>:6: FutureWarning

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]) tensor([280, 280, 280, 280, 280, 280, 280, 280, 280, 280, 280])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]) tensor([70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70])


In [14]:
# 모델, 손실 함수, 옵티마이저 초기화
model = VGG19_Pretrained(num_class=11)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer =optim.Adam(model.parameters(), lr=1e-4)
#optimizer = optim.SGD(model.parameters(), lr=1e-5, momentum=0.9, weight_decay=1e-5)

# def initialize_weights(m):
#     if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#         nn.init.kaiming_normal_(m.weight)
#         if m.bias is not None:
#             nn.init.constant_(m.bias, 0)

# model.apply(initialize_weights)

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

model.apply(initialize_weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

result_dir = Path(root) / 'MK2/Results/VGG19_Pretrained'
result_dir.mkdir(parents=True, exist_ok=True)

# Create directory name.
num_trial=0
parent_dir = result_dir / f'trial_{num_trial}'
while parent_dir.is_dir():
    num_trial = int(parent_dir.name.replace('trial_',''))
    parent_dir = result_dir / f'trial_{num_trial+1}'
parent_dir.mkdir(parents=True, exist_ok=True)
print(f'Trial save path : {parent_dir}')

# 트레이닝 루프
epochs = 15
for epoch in tqdm(range(epochs)):
    print("\n")
    model.train()
    running_loss = 0.0
    train_count = 0
    for inputs, labels in train_loader:

        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_count += inputs.size(0)
        #print(f"Epoch {epoch}/ train_count = {train_count}")

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print(predicted)
            print(labels)

    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")


model_save_path = str(parent_dir) + '/vgg19_pretrained_model.pt'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Trial save path : /gdrive/My Drive/ctp431/MK2/Results/VGG19_Pretrained/trial_5


  0%|          | 0/15 [00:00<?, ?it/s]



Epoch 1/15, Loss: 2.1424
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 8, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 9, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 8, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 6, 0, 0,
        6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
tensor([ 0,  0,  0,  1,  0,  1,  0,  9,  0,  0,  9,  0,  2,  9,  9,  1,  1,  9,
         1, 10,  4,  5,  5,  8,  5,  5,  6,  8,  5,  5,  6,

  7%|▋         | 1/15 [01:05<15:16, 65.44s/it]

tensor([ 6,  4,  5,  5, 10,  5,  5, 10,  5,  4, 10,  4, 10, 10,  5, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  5],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 2.1236, Accuracy: 31.56%


Epoch 2/15, Loss: 1.6232
tensor([ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  7,  0,  1,  1,  8,  8, 10,
         0,  0,  0,  1,  1,  0,  0, 10, 10,  0,  1,  0,  0,  0,  0,  0,  0],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([ 8,  0,  9,  0,  1,  0,  9,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  1,
         0,  1,  0,  0,  9,  0,  0,  0,  0,  0,  1,  0,  0,  2,  0, 10,  8],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

...

 93%|█████████▎| 14/15 [15:12<01:05, 65.18s/it]

tensor([10,  4,  6, 10, 10, 10, 10, 10,  4,  4,  6,  7, 10,  6,  5, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10,  3, 10, 10, 10,  0,  9,  1, 10],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 3.2982, Accuracy: 50.65%


Epoch 15/15, Loss: 0.0861
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 0, 9, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 9, 0, 0, 0, 0, 0, 0, 4, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([1, 1

100%|██████████| 15/15 [16:17<00:00, 65.17s/it]

tensor([10, 10,  5, 10, 10, 10, 10, 10, 10, 10,  6,  2, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10,  9, 10, 10, 10, 10, 10,  7, 10],
       device='cuda:0')
tensor([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       device='cuda:0')
Validation Loss: 2.9561, Accuracy: 57.92%


Model saved to /gdrive/My Drive/ctp431/MK2/Results/VGG19_Pretrained/trial_5/vgg19_pretrained_model.pt
